In [1]:
%load_ext autoreload
%autoreload 2

In [16]:
import pandas as pd
import numpy as np
import sys
sys.path.append("/home/inoue/work/RLRecs/")
from rlrecs.envs.dataset import session_preprocess_data

In [14]:
df = pd.read_csv("/home/inoue/work/dataset/RC15/yoochoose-clicks.dat", sep=",", header=None)
df

/tmp/ipykernel_790187/2431789433.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/home/inoue/work/dataset/RC15/yoochoose-clicks.dat", sep=",", header=None)


,0,1,2,3
0,1,2014-04-07T10:51:09.277Z,214536502,0
1,1,2014-04-07T10:54:09.868Z,214536500,0
2,1,2014-04-07T10:54:46.998Z,214536506,0
3,1,2014-04-07T10:57:00.306Z,214577561,0
4,2,2014-04-07T13:56:37.614Z,214662742,0
...,...,...,...,...
33003939,11299809,2014-09-25T09:33:22.412Z,214819412,S
33003940,11299809,2014-09-25T09:43:52.821Z,214830939,S
33003941,11299811,2014-09-24T19:02:09.741Z,214854855,S
33003942,11299811,2014-09-24T19:02:11.894Z,214854838,S


In [15]:
df.columns = ["sessionId", "timestamp", "itemId", "category"]
df["sessionId"].nunique()

9249729

In [2]:
from rlrecs.envs.dataset import DataLoader

In [1]:
from jax import numpy as jnp
import jax

In [2]:
def recall_at_k(
        y_trues: jnp.ndarray, # (batch_size, )
        y_preds: jnp.ndarray, # (batch_size, k)
        k : int=10
):
    @jax.vmap
    def recall(
        y_trues: jnp.ndarray, # (1, )
        y_preds: jnp.ndarray, # (1, k)
    ):
        y_preds = y_preds[:k]
        return jnp.sum(y_preds == y_trues)
    return jnp.mean(recall(y_trues, y_preds))

In [3]:
def ndcg_at_k(
    y_trues: jnp.ndarray, 
    y_preds: jnp.ndarray,
    k: int=10
):
    @jax.vmap
    def ndcg(
        y_trues: jnp.ndarray,
        y_preds: jnp.ndarray
    ):
        dcg_score = jnp.where(y_trues == y_preds[:k])[0]
        
        return jnp.where(
            dcg_score.shape[0] == 0, 
            0., 
            1. / jnp.log((dcg_score + 2))
        )

    return jnp.mean(ndcg(y_trues, y_preds))

In [4]:
trues = jnp.asarray([1, 2, 4])
preds = jnp.asarray([
    [1, 2, 5],
    [1, 3, 4],
    [2, 4, 5],
])

In [5]:
recall_at_k(trues, preds, k=3)

DeviceArray(0.6666667, dtype=float32)

In [6]:
ndcg_at_k(trues, preds, k=3)

ConcretizationTypeError: Abstract tracer value encountered where concrete value is expected: Traced<ShapedArray(int32[])>with<BatchTrace(level=1/0)> with
  val = DeviceArray([1, 0, 1], dtype=int32)
  batch_dim = 0
The size argument of jnp.nonzero must be statically specified to use jnp.nonzero within JAX transformations.
This BatchTracer with object id 139836015628016 was created on line:
  /tmp/ipykernel_847968/1468760333.py:11 (ndcg)

See https://jax.readthedocs.io/en/latest/errors.html#jax.errors.ConcretizationTypeError